In [ ]:
!wget -q https://raw.githubusercontent.com/dusky04/yes-yes/refs/heads/main/models/__init__.py
!wget -q https://raw.githubusercontent.com/dusky04/yes-yes/refs/heads/main/models/cnn.py
!wget -q https://raw.githubusercontent.com/dusky04/yes-yes/refs/heads/main/models/video_resnet.py
!wget -q https://raw.githubusercontent.com/dusky04/yes-yes/refs/heads/main/dataset.py
!wget -q https://raw.githubusercontent.com/dusky04/yes-yes/refs/heads/main/train.py
!wget -q https://raw.githubusercontent.com/dusky04/yes-yes/refs/heads/main/train_resnet.py
!wget -q https://raw.githubusercontent.com/dusky04/yes-yes/refs/heads/main/utils.py

!mkdir models
!mv __init__.py models/
!mv cnn.py models/
!mv video_resnet.py models/
# maybe just git clone everything

In [ ]:
!pip -q install decord gdown

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 117.2 MB/s eta 0:00:00


In [ ]:
from dataclasses import dataclass

import torch
from torch import nn
from pathlib import Path
from utils import setup_and_download_dataset
from dataset import get_dataloaders
from models.cnn import effnet_b0_model
from torchvision import transforms
from train import train_model


@dataclass
class C:
    LR = 3e-3
    DATASET_NAME = "CricketEC"
    NUM_CLASSES = 14
    NUM_FRAMES = 32
    BATCH_SIZE = 16
    LSTM_HIDDEN_DIM = 64
    LSTM_NUM_LAYERS = 1
    LSTM_DROPOUT = 0.4
    FC_DROPOUT = 0.5
    TRAIN_SIZE = 0.8
    NUM_WORKERS = 1
    PREFETCH_FACTOR = 2
    NUM_EPOCHS = 40
    WEIGHT_DECAY = 3e-4


if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # setup the dataset
    DATASET_NAME = "CricketEC"
    CRICKET_EC_URL = "https://drive.google.com/file/d/1KpKua7OoCjmaQhzF52wdejpGHiDmShZs/view?usp=sharing"

    setup_and_download_dataset(
        DATASET_NAME, url=CRICKET_EC_URL, download_dir=Path("zipped_data")
    )

    c = C()

    # setup transforms
    train_transform = transforms.Compose(
        [
            transforms.ConvertImageDtype(torch.float32),
            transforms.RandomApply([transforms.RandomRotation(15)], p=0.4),
            transforms.RandomApply(
                [transforms.RandomAffine(0, translate=(0.15, 0.15))], p=0.4
            ),
            transforms.RandomApply([transforms.ColorJitter(0.3, 0.3, 0.3, 0.2)], p=0.8),
            transforms.RandomGrayscale(p=0.3),
            transforms.RandomApply(
                [transforms.GaussianBlur(kernel_size=(7, 7))], p=0.5
            ),
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ]
    )
    test_transform = transforms.Compose(
        [
            transforms.ConvertImageDtype(torch.float32),
            transforms.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
            ),
        ]
    )

    # setup dataloaders
    train_dataloader, test_dataloader= get_dataloaders(
        c, train_transform=train_transform, test_transform=test_transform
    )

    # setup model
    model = effnet_b0_model(c).to(device)

    # loss function
    loss_fn = nn.CrossEntropyLoss()

    # optimizer
    # Define parameter groups with different learning rates
    optimizer = torch.optim.Adam([
        {'params': model.feature_extractor.parameters(), 'lr': 1e-4}, # Lower LR for ResNet
        {'params': model.lstm.parameters()},
        {'params': model.linear.parameters()},
        {'params': model.dropout.parameters()}
    ], lr=c.LR, weight_decay = c.WEIGHT_DECAY)

    # lr-scheduler

    # train


Downloading...
From (original): https://drive.google.com/uc?id=1KpKua7OoCjmaQhzF52wdejpGHiDmShZs
From (redirected): https://drive.google.com/uc?id=1KpKua7OoCjmaQhzF52wdejpGHiDmShZs&confirm=t&uuid=1d8046c0-32ab-4d66-ba98-74438ddeeca2
To: /content/zipped_data/CricketEC.zip
100%|██████████| 275M/275M [00:01<00:00, 170MB/s]


Creating directory: /content
Unzipping all the files: 
Unzipping: CricketEC.zip
All files unzipped to: `/content`
Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth


100%|██████████| 20.5M/20.5M [00:00<00:00, 209MB/s]


In [ ]:
train_model(
        c=c,
        exp_name="effenet-32frames-40epochs",
        weights_dir=Path("weights"),
        model=model,
        train_dataloader=train_dataloader,
        test_dataloader=test_dataloader,
        loss_fn=loss_fn,
        optimizer=optimizer,
        scheduler=None,
        device=device,
    )

Processing Epoch: 01:   8%|▊         | 7/88 [01:22<15:51, 11.75s/it, loss=2.8, acc=9.82]


KeyboardInterrupt: 